In [ ]:
# Imports

import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader

In [ ]:
# Define transformations and load datasets
data_transforms = {
    'train': v2.Compose([
        v2.RandomRotation(45),
        v2.RandomResizedCrop(224),
        v2.RandomHorizontalFlip(),
        v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        v2.RandomGrayscale(p=0.1),
        v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        v2.ToTensor(),
        v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': v2.Compose([
        v2.Resize(256),
        v2.CenterCrop(224),
        v2.ToTensor(),
        v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': v2.Compose([
        v2.Resize(256),
        v2.CenterCrop(224),
        v2.ToTensor(),
        v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Specify the data directory
data_path = "data"

# Load the datasets
flowers_train_data = datasets.Flowers102(root=data_path, split='train', download=True, transform=data_transforms['train'])
flowers_val_data = datasets.Flowers102(root=data_path, split='val', download=True, transform=data_transforms['valid'])
flowers_test_data = datasets.Flowers102(root=data_path, split='test', download=True, transform=data_transforms['test'])

# Create data loaders
dataloaders = {
    'train': DataLoader(flowers_train_data, batch_size=128, shuffle=True),
    'val': DataLoader(flowers_val_data, batch_size=128, shuffle=False),
    'test': DataLoader(flowers_test_data, batch_size=128, shuffle=False)
}


dataset_sizes = {
    'train': len(flowers_train_data),
    'val': len(flowers_val_data),
    'test': len(flowers_test_data)
}



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


100%|██████████| 344862509/344862509 [00:10<00:00, 31876194.72it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 860106.46it/s]


100%|██████████| 14989/14989 [00:00<00:00, 22160177.18it/s]


In [ ]:
# Define a Convolutional Neural Network
class FlowerCNN(nn.Module):
    def __init__(self):
        super(FlowerCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.4)

        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.bn6 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 102)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))
        x = self.pool(torch.relu(self.bn5(self.conv5(x))))
        x = x.view(-1, 512 * 7 * 7)
        x = self.dropout(torch.relu(self.bn6(self.fc1(x))))
        x = self.fc2(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Initialise the model

model = FlowerCNN().to(device)

In [ ]:
# Define a Loss Function, Optimizer and Learning Rate Scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)

In [ ]:
# Function to train the network

def train_model(model, criterion, optimizer, scheduler, num_epochs=180):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print(' ' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            phase_start_time = time.time()
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Clear cache to free up memory
                torch.cuda.empty_cache()

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            epoch_time = time.time() - phase_start_time

            print('{} loss: {:.4f} accuracy: {:.4f} time: {:.0f}s'.format(
                phase, epoch_loss, epoch_acc, epoch_time))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training completed in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Validation Accuracy: {:4f}'.format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model




In [ ]:
# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=180)


Epoch 0/180
          
train loss: 4.5242 accuracy: 0.0392 time: 18s
val loss: 4.6271 accuracy: 0.0098 time: 7s

Epoch 1/180
          
train loss: 4.0976 accuracy: 0.1088 time: 15s
val loss: 4.4973 accuracy: 0.0373 time: 7s

Epoch 2/180
          
train loss: 3.8299 accuracy: 0.1490 time: 15s
val loss: 4.0932 accuracy: 0.0765 time: 7s

Epoch 3/180
          
train loss: 3.6674 accuracy: 0.1824 time: 15s
val loss: 3.6850 accuracy: 0.2039 time: 7s

Epoch 4/180
          
train loss: 3.5753 accuracy: 0.2049 time: 15s
val loss: 3.4839 accuracy: 0.2363 time: 7s

Epoch 5/180
          
train loss: 3.5042 accuracy: 0.2147 time: 15s
val loss: 3.4013 accuracy: 0.2412 time: 7s

Epoch 6/180
          
train loss: 3.4167 accuracy: 0.2559 time: 15s
val loss: 3.3689 accuracy: 0.2294 time: 7s

Epoch 7/180
          
train loss: 3.4688 accuracy: 0.2382 time: 15s
val loss: 3.3690 accuracy: 0.2245 time: 7s

Epoch 8/180
          
train loss: 3.4556 accuracy: 0.2186 time: 15s
val loss: 3.3513 accuracy: 

In [ ]:
# Save the trained mode
torch.save(model.state_dict(), 'flowers102_classifier_model.pth')

In [ ]:


# Load the trained model
model.load_state_dict(torch.load('flowers102_classifier_model.pth'))
model.eval()

FlowerCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_fe

In [ ]:
# Function to evaluate the model

def evaluate_model(model, test_loader):
  correct = 0
  total = 0

  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Evaluate the model on the test set
test_accuracy = evaluate_model(model, dataloaders['test'])
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 52.89%


In [ ]:
import torch
torch.cuda.empty_cache()